In [2]:
import pandas as pd
import numpy as np

In [44]:
from os import path
dir_cos = '/home/asigachev/recs-hidden/recs-cos'
dir_als = '/home/asigachev/recs-hidden/recs-als'

In [3]:
col_names = ['user', 'artist-mbid', 'artist-name', 'total-plays']
data = pd.read_csv(
    'lastfm_small.tsv',
    sep='\t',
    header=None,
    names=col_names)

In [4]:
data.headad()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [5]:
data.fillna("None", inplace=True)
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [6]:
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [8]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    test_file_name,
    sep="\t",
    header=False,
    index=False
)
train_file_name = "train/lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    train_file_name,
    sep="\t",
    header=False,
    index=False
)

In [9]:
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

In [12]:
import time
from implicit.nearest_neighbours import CosineRecommender

model_cos = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model_cos.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 1.2766621112823486 секунд


In [43]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(dir_cos, test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 21.19092583656311 секнуд


## ALS

In [24]:
plays2 = plays.astype(np.float64)

In [26]:
from implicit.als import AlternatingLeastSquares

model_als = AlternatingLeastSquares(factors=50)
print("строим матрицу схожести по ALS")
start = time.time()
model_als.fit(plays2)
print("построили матрицу схожести по ALS мере за {} секунд".format(
        time.time() - start))

строим матрицу схожести по ALS
построили матрицу схожести по ALS мере за 12.927585363388062 секунд


In [46]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(dir_als, test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model_als.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 99.7573082447052 секнуд


## Оценка результатов

### kNN

In [45]:
!/home/asigachev/mrec/sbin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /home/asigachev/recs-hidden/lastfm.test.0 \
    --recsdir /home/asigachev/recs-hidden/recs-cos

[2017-11-26 08:13:33,075] INFO: processing /home/asigachev/recs-hidden/lastfm.test.0...
None
mrr            0.0518 +/- 0.0000
prec@5         0.0170 +/- 0.0000
prec@10        0.0152 +/- 0.0000
prec@15        0.0101 +/- 0.0000
prec@20        0.0076 +/- 0.0000


### ALS

In [47]:
!/home/asigachev/mrec/sbin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /home/asigachev/recs-hidden/lastfm.test.0 \
    --recsdir /home/asigachev/recs-hidden/recs-als

[2017-11-26 08:17:14,838] INFO: processing /home/asigachev/recs-hidden/lastfm.test.0...
None
mrr            0.3009 +/- 0.0000
prec@5         0.1302 +/- 0.0000
prec@10        0.1077 +/- 0.0000
prec@15        0.0718 +/- 0.0000
prec@20        0.0539 +/- 0.0000
